In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/files/drug200.csv")
data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data["BP"] = le.fit_transform(data["BP"])
data["Cholesterol"] = le.fit_transform(data["Cholesterol"])
data["Drug"] = le.fit_transform(data["Drug"])
data["Sex"] = le.fit_transform(data["Sex"])

data.dropna(axis = 0)
data

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0,0,25.355,0
1,47,1,1,0,13.093,3
2,47,1,1,0,10.114,3
3,28,0,2,0,7.798,4
4,61,0,1,0,18.043,0
...,...,...,...,...,...,...
195,56,0,1,0,11.567,3
196,16,1,1,0,12.006,3
197,52,1,2,0,9.894,4
198,23,1,2,1,14.020,4


In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category = ConvergenceWarning)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipelines = []
pipelines.append(("ScaledLR", Pipeline([("Scaler", StandardScaler()), ("LR", LogisticRegression())])))
pipelines.append(("ScaledLDA", Pipeline([("Scaler", StandardScaler()), ("LDA", LinearDiscriminantAnalysis())])))
pipelines.append(("ScaledKNN", Pipeline([("Scaler", StandardScaler()), ("KNN", KNeighborsClassifier())])))
pipelines.append(("ScaledCART", Pipeline([("Scaler", StandardScaler()), ("CART", DecisionTreeClassifier())])))
pipelines.append(("ScaledNB", Pipeline([("Scaler", StandardScaler()), ("NB", GaussianNB())])))
pipelines.append(("ScaledSVM", Pipeline([("Scaler", StandardScaler()), ("SVM", SVC(max_iter = 300))])))

# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Drug', axis = 1)
Y = data['Drug']
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train, y_train = SMOTE.fit_resample(X_train, y_train)
# summarize class distribution
print("After oversampling: ", Counter(y_train))
print("-" * 80)

# Evaluate each model
results = []
names = []
metrics = ["accuracy", "precision", "recall", "f1", "roc_auc"]

for metric in metrics:
   print(metrics[metrics.index(metric)] + ": \n")
   for name, model in pipelines:
       kfold = KFold(n_splits = 10, random_state = 7, shuffle = True)
       cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = metric)
       results.append(cv_results)
       names.append(name)
       msg = (name, cv_results.mean(), cv_results.std( ))
       print(msg)
   print("-" * 80)

Before oversampling:  Counter({0: 76, 4: 43, 1: 17, 2: 13, 3: 11})
After oversampling:  Counter({4: 76, 0: 76, 3: 76, 1: 76, 2: 76})
--------------------------------------------------------------------------------
accuracy: 

('ScaledLR', 0.9763157894736842, 0.024826266136991088)
('ScaledLDA', 0.9157894736842105, 0.04527539614232963)
('ScaledKNN', 0.9184210526315789, 0.038044295512634094)
('ScaledCART', 0.9947368421052631, 0.010526315789473674)
('ScaledNB', 0.9236842105263158, 0.036178229170704)
('ScaledSVM', 0.9736842105263156, 0.026315789473684226)
--------------------------------------------------------------------------------
precision: 



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1954, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1573, in precision_recall_fscore_support
    labels = _check_set_wise_labels(y_true, y_pred, average, labels, pos_label)
  File "/usr/local/lib/python3.10/dist-packages/sk

('ScaledLR', nan, nan)
('ScaledLDA', nan, nan)
('ScaledKNN', nan, nan)
('ScaledCART', nan, nan)
('ScaledNB', nan, nan)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1954, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1573, in precision_recall_fscore_support
    labels = _check_set_wise_labels(y_true, y_pred, average, labels, pos_label)
  File "/usr/local/lib/python3.10/dist-packages/sk

('ScaledSVM', nan, nan)
--------------------------------------------------------------------------------
recall: 

('ScaledLR', nan, nan)
('ScaledLDA', nan, nan)


...

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 2098, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1573, in precision_recall_fscore_support
    labels = _check_set_wise_labels(y_true, y_pred, average, labels, pos_label)
  File "/usr/local/lib/python3.10/dist-packages/sklea

('ScaledLR', nan, nan)
('ScaledLDA', nan, nan)
('ScaledKNN', nan, nan)
('ScaledCART', nan, nan)
('ScaledNB', nan, nan)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1573, in precision_recall_fscore_support
  

('ScaledSVM', nan, nan)
--------------------------------------------------------------------------------
roc_auc: 

('ScaledLR', nan, nan)
('ScaledLDA', nan, nan)
('ScaledKNN', nan, nan)
('ScaledCART', nan, nan)
('ScaledNB', nan, nan)
('ScaledSVM', nan, nan)
--------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 367, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cach

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

In [ ]:
# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Drug', axis = 1)
Y = data['Drug']
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train,y_train= SMOTE.fit_resample(X_train,y_train)
# summarize class distribution
print("After oversampling: ", Counter(y_train))
print("-" * 80)

# Prepare Models
models = []
models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))
models.append(('GBM', GradientBoostingClassifier()))
models.append(('AB', AdaBoostClassifier()))

metrics = ["accuracy", "precision", "recall", "f1", "roc_auc"]

for metric in metrics:
   print(metrics[metrics.index(metric)] + ": \n")
   for name, model in models:
       kfold = KFold(n_splits = 10, random_state = 7, shuffle = True)
       cv_results = cross_val_score(model,X_train,y_train, cv = kfold, scoring = metric)
       msg = (name, cv_results.mean(), cv_results.std( ))
       print(msg)
   print("-" * 80)

Before oversampling:  Counter({0: 76, 4: 43, 1: 17, 2: 13, 3: 11})
After oversampling:  Counter({4: 76, 0: 76, 3: 76, 1: 76, 2: 76})
--------------------------------------------------------------------------------
accuracy: 

('RF', 0.9973684210526315, 0.007894736842105255)
('ET', 0.9842105263157894, 0.01745591994923895)
('GBM', 1.0, 0.0)
('AB', 0.5157894736842106, 0.06023959548557682)
--------------------------------------------------------------------------------
precision: 



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1954, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1573, in precision_recall_fscore_support
    labels = _check_set_wise_labels(y_true, y_pred, average, labels, pos_label)
  File "/usr/local/lib/python3.10/dist-packages/sk

('RF', nan, nan)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1954, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 1573, in precision_recall_fscore_support
    labels = _check_set_wise_labels(y_true, y_pred, average, labels, pos_label)
  File "/usr/local/lib/python3.10/dist-packages/sk

('ET', nan, nan)


...

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 367, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cach

('GBM', nan, nan)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 367, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cach

('AB', nan, nan)
--------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 367, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cach

In [ ]:
# Voting classifier
from sklearn.ensemble import VotingClassifier
# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Drug', axis=1)
Y = data['Drug']
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train, y_train = SMOTE.fit_resample(X_train, y_train)

print("After oversampling: ", Counter(y_train))
print("-" * 80)

rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
et = ExtraTreesClassifier()
# Fit the models
clfvt = VotingClassifier(estimators = [('rf',rf),('gb', gb),('et', et)], voting = "soft")
clfvt.fit(X_train,y_train)

# Predict test data

y_predvt = clfvt.predict(X_test)

print("The accuracy_score of logit model is:\n", accuracy_score(y_test, y_predvt))
print("-" * 80)
print("The classification report of logit model is:\n", classification_report(y_test, y_predvt))
print("-" * 80)
print("The confusion_matrix of logit model is:\n", confusion_matrix(y_test, y_predvt))
print("-" * 80)
#print("The roc_auc_score of logit model is:\n", roc_auc_score(y_test, y_predvt))

Before oversampling:  Counter({0: 76, 4: 43, 1: 17, 2: 13, 3: 11})
After oversampling:  Counter({4: 76, 0: 76, 3: 76, 1: 76, 2: 76})
--------------------------------------------------------------------------------
The accuracy_score of logit model is:
 1.0
--------------------------------------------------------------------------------
The classification report of logit model is:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00        11

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

--------------------------------------------------------------------------------
The confusion_matrix of logit model is:
 [[15 

In [ ]:
#Train other classifiers
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score

# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter

gb = GradientBoostingClassifier()

X = data.drop('Drug', axis=1)
Y = data['Drug']
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)
# Summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train, y_train = SMOTE.fit_resample(X_train, y_train)

print("After oversampling: ", Counter(y_train))
print("-" * 80)

grid_space = dict(n_estimators = np.array([50, 100, 150, 200, 250, 300, 350, 400]))

from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(gb, param_grid = grid_space, cv = 3, scoring = 'accuracy')
model_grid = grid.fit(X, Y)

print('Best hyperparameters are: '+str(model_grid.best_params_))
print('Best score is: '+str(model_grid.best_score_))

Before oversampling:  Counter({0: 76, 4: 43, 1: 17, 2: 13, 3: 11})
After oversampling:  Counter({4: 76, 0: 76, 3: 76, 1: 76, 2: 76})
--------------------------------------------------------------------------------
Best hyperparameters are: {'n_estimators': 50}
Best score is: 0.9849238655208804


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score

# Import resampling libraries
from imblearn.over_sampling import SMOTE
from collections import Counter
# Load data
X = data.drop('Drug', axis=1)
Y = data['Drug']
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

# summarize class distribution
print("Before oversampling: ", Counter(y_train))
# define oversampling strategy
SMOTE = SMOTE()
# fit and apply the transform
X_train, y_train = SMOTE.fit_resample(X_train, y_train)
# summarize class distribution
print("After oversampling: ", Counter(y_train))
print("-" * 80)
# Train the model
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
# Predicting the model
y_pred = model.predict(X_test)

print("The classification report of logit model is:\n", classification_report(y_test, y_pred))
print("-" * 80)
print("The accuracy_score of logit model is:\n", accuracy_score(y_test, y_pred))
print("-" * 80)
print("The confusion_matrix of logit model is:\n", confusion_matrix(y_test, y_pred))
print("-" * 80)
#print("The roc_auc_score of logit model is:\n", roc_auc_score(y_test, y_pred))

Before oversampling:  Counter({0: 76, 4: 43, 1: 17, 2: 13, 3: 11})
After oversampling:  Counter({4: 76, 0: 76, 3: 76, 1: 76, 2: 76})
--------------------------------------------------------------------------------
The classification report of logit model is:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00        11

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

--------------------------------------------------------------------------------
The accuracy_score of logit model is:
 1.0
--------------------------------------------------------------------------------
The confusion_matrix of logit model is:
 [[15 

ValueError: multi_class must be in ('ovo', 'ovr')